In [1]:
import SimpleITK as sitk
import os
import pandas as pd
import json
import numpy as np
import glob

In [2]:
# data path
input_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'
output_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'

In [3]:
# Read / write SWC files
# To do - move this to a library
scols = ["Index","Type","X","Y","Z","Radius","Parent"]
sdict = {}
for idx in range(len(scols)) :
    sdict[idx] = scols[idx]

def read_swc (fname) :
    
    df = pd.read_csv( fname, sep=r"\s+", comment = "#", header=None )
    
    # keep only the 7 standard columns
    dcols = df.columns[len(scols):]
    df = df.drop( dcols, axis = 1 )
    df = df.rename( columns = sdict )
    df.set_index('Index', inplace=True)
    
    return df

def write_swc (df, fname) :
    df.to_csv(fname,sep = ' ', header=False)

In [4]:
# open combined and downsampled inventory files
combined_file = 'combined_inventory.csv'

In [5]:
samples = pd.read_csv(os.path.join(input_directory,combined_file))
samples.set_index('HUST Sample ID', inplace=True)

In [ ]:
# mapping from rotation value in json to orientation
rotation = {}
rotation[0] = 'SLA'

imgext = '.nii.gz'
swcext = '.swc'

#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}



In [6]:
samples = samples.loc[[17543,182725]]

In [7]:
# iterate for each sample

for s, row in samples.iterrows() :
    
    print(s)
    
    dname = row['Sample ID']
    
    #
    # create output downsampled directory
    #
    odir = os.path.join( output_directory, dname, 'overview')
    if not os.path.exists( odir ) :
        os.makedirs( odir )

    #
    # open json metadata file
    #
    if pd.isna(row['JSON Files']) :
        continue
        
    jname = row['JSON Files'].split(';')[0]
    jpath = os.path.join( input_directory, dname, jname )
    
    with open(jpath,) as jf :
        jdata = json.load(jf)
    
    otag = rotation[jdata['SampleInfo']['Rotation']]
    
    #
    # open the green downsampled image
    #
    idir = os.path.join( input_directory, dname, 'downsampled' )
    
    for ifile in glob.glob(os.path.join(idir, "*_green_*%s" % imgext) ) :
        print(" : " +  ifile)
        volume = sitk.ReadImage( ifile )
        break
        
    #
    # iterate swc files in directory
    #
    rdir = os.path.join( input_directory, dname, 'morphology' )
    
    for rfile in glob.glob(os.path.join(rdir,'*.swc') ) :
        
        print("  : " +  rfile)
        
        # open swc file as pandas frame
        ndf = read_swc(rfile)
        break
    
    break

        
    

17543
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\mouseID_325875-17543\downsampled\17543_green_0.0112x0.0112x0.0080_mm_SLA.nii.gz
  : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\mouseID_325875-17543\morphology\1059283974_17543_3828-X6023-Y17044_um_SLA.swc


In [8]:
ndf

,Type,X,Y,Z,Radius,Parent
Index,,,,,,
2008285,2,2383.5000,5304.95000,3989.00,1.0,2008286
2008286,2,2383.5000,5306.52500,3989.14,1.0,2008287
2008287,2,2383.5000,5308.27500,3989.10,1.0,2008288
2008288,2,2383.5000,5309.88500,3990.16,1.0,2008289
2008289,2,2383.5000,5311.56500,3991.00,1.0,2008290
...,...,...,...,...,...,...
2013010,2,2272.5780,6358.62500,3846.80,1.0,2013011
2013003,2,2269.1165,6368.39000,3835.37,1.0,2013004
2013002,2,2268.4725,6369.86000,3834.72,1.0,2013003


In [9]:
print(volume)

Image (000002690F1E9CE0)
  RTTI typeinfo:   class itk::Image<unsigned short,3>
  Reference Count: 1
  Modified Time: 879
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 856
  UpdateMTime: 878
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [591, 787, 1277]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [591, 787, 1277]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [591, 787, 1277]
  Spacing: [0.0112, 0.0112, 0.008]
  Origin: [8.8032, 0, 6.608]
  Direction: 
-0 -1 0
0 -0 1
-1 0 0

  IndexToPointMatrix: 
0 -0.0112 0
0 0 0.008
-0.0112 0 0

  PointToIndexMatrix: 
0 0 -89.2857
-89.2857 0 0
0 125 0

  Inverse Direction: 
0 0 -1
-1 0 0
0 1 0

  PixelContainer: 
    ImportImageContainer (000002690E0831E0)
      RTTI typeinfo:   class itk::ImportImageContainer<unsig